In [1]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
import torchvision.utils as vutils




In [2]:
% load_ext autoreload 
% autoreload 2

In [3]:
from modules_tied import VAE
from modules_tied import NetD
from modules_tied import Aux
from modules_tied import loss_function

In [4]:
bsz = 100
criterion = nn.BCELoss()


In [5]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=bsz, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=bsz, shuffle=True)


In [6]:
netG = VAE()
netD = NetD()
#aux = Aux()
print(netG)

VAE(
  (fc1): Linear(in_features=784, out_features=400, bias=True)
  (fc21): Linear(in_features=400, out_features=20, bias=True)
  (fc22): Linear(in_features=400, out_features=20, bias=True)
  (fc3): Linear(in_features=20, out_features=400, bias=True)
  (fc4): Linear(in_features=400, out_features=784, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
  (mu_): Sequential(
    (0): Conv2d(1, 8, kernel_size=(5, 5), stride=(2, 2), bias=False)
    (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): Conv2d(8, 64, kernel_size=(5, 5), stride=(2, 2), bias=False)
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): Conv2d(64, 20, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (7): ReLU(inplace)
  )
  (logsigma_): Sequential(
    (0): Conv2d(1, 8, kernel_size=(5, 5), stride=(2, 2), bias=False)
    (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_r

In [7]:
optimizerD = optim.Adam(netD.parameters(), lr=1e-3)
optimizerG = optim.Adam(netG.parameters(), lr=1e-3)
#optimizer_aux = optim.Adam(aux.parameters(), lr=1e-4)

In [8]:
input = torch.FloatTensor(bsz,28,28)
label = torch.FloatTensor(bsz)
real_label=1
fake_label=0
USE_CUDA=1
lamb = 1e-1
l1dist = nn.PairwiseDistance(1)
l2dist = nn.PairwiseDistance(2)
LeakyReLU = nn.LeakyReLU(0)

if(USE_CUDA):
    netG=netG.cuda()
    netD=netD.cuda()
    #aux = aux.cuda()
    #criterion=criterion.cuda()
    input,label=input.cuda(), label.cuda()
    l1dist = l1dist.cuda()
    l2dist = l2dist.cuda()
    LeakyReLU = LeakyReLU.cuda()

In [9]:
def get_direct_gradient_penalty(netD, x, gamma, cuda):
    _,output = netD(x)
    gradOutput = torch.ones(output.size()).cuda() if cuda else torch.ones(output.size())
    
    gradient = torch.autograd.grad(outputs=output, inputs=x, grad_outputs=gradOutput, create_graph=True, retain_graph=True, only_inputs=True)[0]
    gradientPenalty = (gradient.norm(2, dim=1)).mean() * gamma
    
    return gradientPenalty


In [10]:
from train import run_trainer

In [ ]:
run_trainer(netG,netD,optimizerG,optimizerD,train_loader,bsz)

epoch  0
epoch  0
epoch  0
epoch  0
epoch  0
epoch  0
losses 0 4593.00986433 70135.7931023
e_enc [70135.793102264404]
epoch  1
epoch  1
epoch  1
epoch  1
epoch  1
epoch  1
losses 1 3396.77171326 118242.238358
e_enc [70135.793102264404, 118242.23835754395]
epoch  2
epoch  2
epoch  2
epoch  2
epoch  2
epoch  2
losses 2 3383.91936255 145650.230942
e_enc [70135.793102264404, 118242.23835754395, 145650.23094177246]
epoch  3
epoch  3
epoch  3
epoch  3
epoch  3
epoch  3
losses 3 3459.2256813 168377.874924
e_enc [70135.793102264404, 118242.23835754395, 145650.23094177246, 168377.87492370605]
epoch  4
epoch  4
epoch  4
epoch  4
epoch  4
epoch  4
losses 4 3542.97210598 185323.607452
e_enc [70135.793102264404, 118242.23835754395, 145650.23094177246, 168377.87492370605, 185323.60745239258]
epoch  5
epoch  5
epoch  5
epoch  5
epoch  5
epoch  5
losses 5 3555.69479704 196563.170929
e_enc [70135.793102264404, 118242.23835754395, 145650.23094177246, 168377.87492370605, 185323.60745239258, 196563.170928

epoch  24
epoch  24
epoch  24
epoch  24
epoch  24
losses 24 3610.70515203 272452.373993
e_enc [70135.793102264404, 118242.23835754395, 145650.23094177246, 168377.87492370605, 185323.60745239258, 196563.17092895508, 208341.9807434082, 217114.25103759766, 223238.54135131836, 230004.33187866211, 234951.48391723633, 241461.49420166016, 241890.1318359375, 248013.57397460938, 254795.72735595703, 258794.75241088867, 262246.67562866211, 265505.37365722656, 264079.98022460938, 263619.64297485352, 265594.72317504883, 264764.32614135742, 267404.25018310547, 269964.87448120117, 272452.37399291992]
epoch  25
epoch  25
epoch  25
epoch  25
epoch  25
epoch  25
losses 25 3638.4140296 275170.362335
e_enc [70135.793102264404, 118242.23835754395, 145650.23094177246, 168377.87492370605, 185323.60745239258, 196563.17092895508, 208341.9807434082, 217114.25103759766, 223238.54135131836, 230004.33187866211, 234951.48391723633, 241461.49420166016, 241890.1318359375, 248013.57397460938, 254795.72735595703, 25879

epoch  36
epoch  36
epoch  36
epoch  36
epoch  36
epoch  36
losses 36 3538.12281036 289051.889832
e_enc [70135.793102264404, 118242.23835754395, 145650.23094177246, 168377.87492370605, 185323.60745239258, 196563.17092895508, 208341.9807434082, 217114.25103759766, 223238.54135131836, 230004.33187866211, 234951.48391723633, 241461.49420166016, 241890.1318359375, 248013.57397460938, 254795.72735595703, 258794.75241088867, 262246.67562866211, 265505.37365722656, 264079.98022460938, 263619.64297485352, 265594.72317504883, 264764.32614135742, 267404.25018310547, 269964.87448120117, 272452.37399291992, 275170.36233520508, 278563.48526000977, 283019.06900024414, 245935.79260253906, 276847.337890625, 277989.42886352539, 276217.47415161133, 277613.5393371582, 283180.42825317383, 286904.72125244141, 287642.78115844727, 289051.88983154297]
epoch  37
epoch  37
epoch  37
epoch  37
